## Import libraries

In [3]:
import sys
sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
from sem_covid.services.store_registry import store_registry
from sem_covid import config

## Define constants

In [9]:
DOCUMENT_EMBEDDINGS_FEATURE_STORE_NAME = 'fs_doc_emb_tfidf'

In [2]:
es_store = store_registry.es_index_store()

In [24]:
configs = [config.PWDB_ELASTIC_SEARCH_INDEX_NAME,
           config.EU_CELLAR_ELASTIC_SEARCH_INDEX_NAME,
           config.EU_TIMELINE_ELASTIC_SEARCH_INDEX_NAME,
           config.IRELAND_TIMELINE_ELASTIC_SEARCH_INDEX_NAME
           ]

In [25]:
df_list = [es_store.get_dataframe(index_name=index) for index in configs]

100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (2818 of 2818) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (171 of 171) |######################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1859 of 1859) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [28]:
for df in df_list:
    print(df.columns)

Index(['identifier', 'title', 'title_national_language', 'country',
       'start_date', 'end_date', 'date_type', 'type_of_measure',
       'status_of_regulation', 'category', 'subcategory', 'creation_date',
       'background_info_description', 'content_of_measure_description',
       'use_of_measure_description', 'actors', 'target_groups', 'funding',
       'involvement_of_social_partners_description',
       'social_partner_involvement_form', 'social_partner_role',
       'is_sector_specific', 'private_or_public_sector',
       'is_occupation_specific', 'sectors', 'occupations', 'sources'],
      dtype='object')
Index(['work', 'title', 'cdm_types', 'cdm_type_labels', 'resource_types',
       'resource_type_labels', 'eurovoc_concepts', 'eurovoc_concept_labels',
       'subject_matters', 'subject_matter_labels', 'directory_codes',
       'directory_codes_labels', 'celex_numbers', 'legal_elis', 'id_documents',
       'same_as_uris', 'authors', 'author_labels', 'full_ojs', 'oj_sectors',

In [27]:
df_list[0].start_date

_id
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0    10/20/2020
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc    05/06/2020
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353    08/25/2020
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044    03/01/2020
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b    01/01/2021
                                                                       ...    
cb014a456b14c3621dd318a12e611f70c2a9636be9fe181072bd4bf5917a40fa    03/18/2020
d233b17dc2b98f14269c2b22be78d93ec5ccf2a0013b86f09175c69353c5800b    05/20/2020
77d7e3c52aaf78bdfb1a1667641db1293bbff862440c547fc3f6e8ab4fbd0d4e    03/17/2020
be8e21b382b63dd838087d7864e4d49a4807d9cc5d71aaa82577383ece92c581    07/01/2020
c434820f8f8c0b1447ba2f6f62aed177b9597532d7fcd156c6f556532dd7d11e    09/03/2020
Name: start_date, Length: 1288, dtype: object

In [17]:
import time
from datetime import datetime

In [21]:
df.updated_date

_id
003ea2a9313db86a3e0b9cd272d4dd861ed92cfab3a5d120725211cc113b1f93    17 February 2021
004a77d6db5f3be729b4b3d8b7eb0ccb564defb037a0edcba96eb5c164d87a37       22 April 2021
00ab798bb7ed96ce3b93b9a3488522a58853bcaaf1bfa1bcf05fd8ffd8af0ce1    18 December 2020
00abfb8547a6cfc95a4591e489431fd7803cbb8f51db632db1e462096e914c75        5 March 2021
01361189adf68b9a6d91beffd9f0b8259d5764ba865aab9dae3982545deeeb8f    16 November 2020
                                                                          ...       
fddf8d1e3ce04ef12941ac7a83a2e85806c84d83496f41b0cc6d98bb0d8910e8     30 January 2021
fde72094e795e2f9a055d1c3fc581985e028ddcaeb436c1d5056f109c6e853d0       15 March 2021
fefa88e8bcb8f19c1794526db7e241a1201a7aa6f05fedd25c09cee4b35c46f0         18 May 2020
ff474a2f55ba0a59240e4e986870ba12be9f2238bec4f4f8ca2c9c712172ed43     8 February 2021
ff4ea2ef933287c4125d898245e878ae9fcb1b356b3c60de288dd97020bdd3a5     23 January 2021
Name: updated_date, Length: 846, dtype: object

In [22]:
df.updated_date.apply(lambda x: datetime.strptime(x,'%d %B %Y')).sort_values()

_id
bda5b336f5422961710173eb8d3410f9a7de84730384e5a0a9e4bca7944d8ef2   2020-02-06
3081ea5944895377e13ba7099afd23af5e5e6758353487c34b5b66a7208eb320   2020-02-11
07e10eabe926db01330fe75c8e24204d2b2b0f7b452b22a9819bd942f589068a   2020-02-11
f16d3a88f070b5bffcd6468060b7cd65aec942fdb7bed206554a35e7397f59f3   2020-02-18
48d020cbb50ca48a690b248341563ef320baa2dc0c733c8f46d4f9e99e0a0109   2020-02-26
                                                                      ...    
2ef0d1006a3ae21968f4805e4427a93f33dea7bd576dfa2b1d51c721dbbce336   2021-07-30
840f37b71ae8997f9708ee15e84424323edb8133fd13df26fa16b5fdd444c15a   2021-08-03
7b90e6440a9e2bd26a05f6f56f49dd7c80814e43480854be1debe461c63bdbb7   2021-08-04
ef6430aadbb96ea6654b296f6ba51a6d450a398bc6e5e7d9262f60a4fc3a6a2a   2021-08-05
f6adde50065d7eb9bbc6ee2585c90c5e90810aba4d966ceb3282bf0961aef0a3   2021-08-10
Name: updated_date, Length: 846, dtype: datetime64[ns]

In [10]:
doc_emb = es_store.get_dataframe(index_name=DOCUMENT_EMBEDDINGS_FEATURE_STORE_NAME)

100% (2490 of 2490) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [16]:
doc_emb.columns

Index(['text', 'source', 'segment_type', 'segment_id', 'prepare_method',
       'textual_columns', 'embedding', 'embedding_method'],
      dtype='object')